[View in Colaboratory](https://colab.research.google.com/github/krishna-sharma19/sparcs/blob/master/notebook.ipynb)

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import preprocessing    
import tensorflow as tf
from SALib.analyze import delta
from SALib.util import read_param_file
import numpy as np


def cleanAndProcessData():
    patient_data_df = pd.read_csv('../input/Hospital_Inpatient_Discharges__SPARCS_De-Identified___2015.csv')
    X_Full = patient_data_df[['Age Group', 'Gender', 'Race', 'Ethnicity', 'Type of Admission','CCS Diagnosis Code','CCS Procedure Code',  'APR DRG Code', 'APR MDC Code','APR Severity of Illness Code','APR Risk of Mortality']]
    pd.options.mode.chained_assignment = None
    X_Full['Age Group']=pd.factorize(X_Full['Age Group'])[0]
    X_Full['APR Risk of Mortality']=pd.factorize(X_Full['APR Risk of Mortality'])[0]
    X_Full=pd.get_dummies(X_Full, columns=["Race"])
    X_Full=pd.get_dummies(X_Full, columns=["Ethnicity"])
    X_Full=pd.get_dummies(X_Full, columns=["Type of Admission"])
    X_Full=pd.get_dummies(X_Full, columns=["Gender"])
    Y_Full = patient_data_df[['Length of Stay']]
    Y_Full = Y_Full.replace('120 +', 120)
    Y_Full['Length of Stay'] = Y_Full['Length of Stay'].apply(pd.to_numeric)

    Si = delta.analyze(problem, X_Full, Y_Full, num_resamples=10, conf_level=0.95, print_to_console=False)
    print(str(Si['delta']))
    return (patient_data_df,X_Full,Y_Full)

In [0]:
def randomSample(patient_data_df,X_Full,Y_Full):
    msk = np.random.rand(len(patient_data_df)) < 0.8
    X_training = X_Full[msk]
    X_testing = X_Full[~msk]

    Y_training = Y_Full[msk]
    Y_testing = Y_Full[~msk]

    from sklearn.preprocessing import MinMaxScaler
    import tensorflow as tf

    # tf.reset_default_graph()
    X_scaler = MinMaxScaler(feature_range=(0, 1))
    Y_scaler = MinMaxScaler(feature_range=(0, 1))

    # Scale both the training inputs and outputs
    X_scaled_training = X_scaler.fit_transform(X_training)
    Y_scaled_training = Y_scaler.fit_transform(Y_training)

    # It's very important that the training and test data are scaled with the same scaler.
    X_scaled_testing = X_scaler.transform(X_testing)
    Y_scaled_testing = Y_scaler.transform(Y_testing)
    return (Y_scaler,Y_testing,(X_scaled_training,Y_scaled_training),(X_scaled_testing ,Y_scaled_testing))


In [0]:
def neuralTraining(patient_data_df,X,Y,learning_rate = 0.001,training_epochs = 100, layer_1_nodes = 100, layer_2_nodes = 250, layer_3_nodes = 250 ,layer_4_nodes = 100):
    (Y_scaler,Y_testing,(X_scaled_training, Y_scaled_training), (X_scaled_testing, Y_scaled_testing)) = randomSample(patient_data_df,X,Y)
    # Define how many inputs and outputs are in our neural network
    number_of_inputs = 24
    number_of_outputs = 1

    # Section One: Define the layers of the neural network itself

    # Input Layer
    tf.reset_default_graph() 
    with tf.variable_scope('input'):
        X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

    # Layer 1
    with tf.variable_scope('layer_1'):
        weights = tf.get_variable("weights1", shape=[number_of_inputs, layer_1_nodes],
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
        layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

    # Layer 2
    with tf.variable_scope('layer_2'):
        weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes],
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
        layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

    # Layer 3
    with tf.variable_scope('layer_3'):
        weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes],
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
        layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

    # Layer 4
    with tf.variable_scope('layer_4'):
        weights = tf.get_variable("weights4", shape=[layer_3_nodes, layer_4_nodes],
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases4", shape=[layer_4_nodes], initializer=tf.zeros_initializer())
        layer_4_output = tf.nn.relu(tf.matmul(layer_3_output, weights) + biases)

    # Output Layer
    with tf.variable_scope('output'):
        weights = tf.get_variable("weights5", shape=[layer_4_nodes, number_of_outputs],
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable(name="biases5", shape=[number_of_outputs], initializer=tf.zeros_initializer())
        prediction = tf.matmul(layer_4_output, weights) + biases

    # Section Two: Define the cost function of the neural network that will measure prediction accuracy during training

    with tf.variable_scope('cost'):
        Y = tf.placeholder(tf.float32, shape=(None, 1))
        cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

    # Section Three: Define the optimizer function that will be run to optimize the neural network

    with tf.variable_scope('train'):
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # Create a summary operation to log the progress of the network
    with tf.variable_scope('logging'):
        tf.summary.scalar('current_cost', cost)
        summary = tf.summary.merge_all()

    rmsds= []
    arr = np.asarray(Y_testing['Length of Stay'])
    arr = np.transpose(np.asmatrix(arr))

    saver = tf.train.Saver()
    with tf.Session() as session:
        # When loading from a checkpoint, don't initialize the variables!
        session.run(tf.global_variables_initializer())
        for epoch in range(training_epochs):

            # Feed in the training data and do one step of neural network training
            session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

            # Every 5 training steps, log our progress
            if epoch % 3 == 0:
                # Get the current accuracy scores by running the "cost" operation on the training and test data sets
                training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
                testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

                Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})

                Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

                rmsd = np.sqrt(np.mean(np.asarray((arr - Y_predicted)) ** 2))
                Y_predicted = Y_predicted.astype(np.int64, copy=False)
                import csv
                with open(r'stats.cxv', 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow([learning_rate,training_epochs, layer_1_nodes, layer_2_nodes, layer_3_nodes  ,layer_4_nodes,rmsd])


                # Print the current training status to the screen
                print("Epoch: {} - Training Cost: {}  Testing Cost: {} RMSD: {}".format(epoch, training_cost, testing_cost, rmsd))

            # Training is now complete!

            # Get the final accuracy scores by running the "cost" operation on the training and test data sets
            final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
            final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

        print("Final Training cost: {}".format(final_training_cost))
        print("Final Testing cost: {}".format(final_testing_cost))

        # print(arr)
        # print(Y_predicted)
        import matplotlib.pyplot as plt

        # np.histogram(arr-Y_predicted)
        plt.hist(arr - Y_predicted)
        plt.show()
        import csv
        with open(r'stats.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow([learning_rate,training_epochs, layer_1_nodes, layer_2_nodes, layer_3_nodes  ,layer_4_nodes,rmsd])


patient_data_df,X,Y = cleanAndProcessData();
# for i in range(25,500,25):
#     for j in range(i,500,25):
#         for k in range(j,500,25):
#             for l in range(25,j,25):
#                 print("layer 1 nodes",i,j,k,l)
#                 neuralTraining(patient_data_df,X,Y,learning_rate = 0.01,training_epochs = 15, layer_1_nodes = 100, layer_2_nodes = 150, layer_3_nodes = 150 ,layer_4_nodes = 100)
# import sys



# sys.path.append('../..')


# Read the parameter range file and generate samples
# Since this is "given data", the bounds in the parameter file will not be used
# but the columns are still expected
# problem = read_param_file('../../SALib/test_functions/params/Ishigami.txt')
# X = np.loadtxt('model_input.txt')
# Y = np.loadtxt('model_output.txt')

# Perform the sensitivity analysis using the model output
# Specify which column of the output file to analyze (zero-indexed)
# Si = delta.analyze(problem, X, Y, num_resamples=10, conf_level=0.95, print_to_console=False)
# Returns a dictionary with keys 'delta', 'delta_conf', 'S1', 'S1_conf'
# print(str(Si['delta']))

